In [1]:
import folium
import folium.plugins
import pandas as pd

In [2]:
df = pd.read_csv('../data/MERGED2015_16_PP.csv', low_memory=False)

In [3]:
# narrow data

schools = df[['UNITID',           # institution ID
              'INSTNM',           # institution name
              'CITY',             # institution city
              'STABBR',           # institution state abbrev
              'ZIP',              # institution zip
              'PREDDEG',          # predominate degree
              'CURROPER',         # currently operating flag
              'TUITIONFEE_IN',    # in-state tuition and fees
              'LATITUDE',         # latitude
              'LONGITUDE',        # longitude
              'DISTANCEONLY',     # distance only flag
              'DEATH_YR4_RT',     # student death 
              'COMP_ORIG_YR4_RT', # student completion
              'GRAD_DEBT_MDN'     # median debt
             ]]

In [4]:
schools['info'] = schools.apply(lambda x: '{}<br>{}, {} {}<br>Median debt: ${}'.format(
    x['INSTNM'], x['CITY'], x['STABBR'], x['ZIP'], x['GRAD_DEBT_MDN']).replace("'", r'\u0027'), axis=1)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
# filter data
schools = schools[(schools['PREDDEG'] == 3) &                                   # Predominate degree is BS
                  (schools['CURROPER'] == 1) &                                  # Currently operating
                  (schools['DISTANCEONLY'] == 0) &                              # Not distance
                  (schools['TUITIONFEE_IN'].notnull()) &                        # Key measurements aren't missing
                  (schools['LATITUDE'].notnull()) &
                  (schools['LONGITUDE'].notnull()) &
                  (schools['LATITUDE'] > 20) & (schools['LATITUDE'] < 50) &     # Location is US 48
                  (schools['LONGITUDE'] > -130) & (schools['LONGITUDE'] < -60)]

In [6]:
m = folium.Map(location=[42.0285, -93.65], zoom_start=4)
icon_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Emoji_u1f4b0.svg/1024px-Emoji_u1f4b0.svg.png'

locations = list(schools.apply(lambda x: [x['LATITUDE'], x['LONGITUDE']], axis=1))
popups = list(schools['info'])
icons = [folium.features.CustomIcon(icon_url, icon_size=(30, 30)) for i in range(schools.shape[0])]

marker_cluster = folium.plugins.MarkerCluster(locations=locations, popups=popups, icons=icons).add_to(m)

In [7]:
m

In [8]:
m.save(outfile='../docs/map.html')